# 추천시스템 만들기
* MovieLens 1M Dataset

|평가문항|상세기준|
|---|---|
|1. CSR matix가 정상적으로 만들어졌다|사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다|
|2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|
|3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.|MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.|

# 라이브러리 및 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np
import os

from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

# 1) 데이터 준비와 전처리

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
print(orginal_data_size)
ratings.head()

1000209


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
# 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿈
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터 불러오기
movie_file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


# 2) 분석
* ratings에 있는 유니크한 영화 개수
* ratings에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

In [6]:
ratings.nunique()

user_id        6039
movie_id       3628
count             3
timestamp    412911
dtype: int64

In [7]:
ratings['movie_id'].nunique()

3628

* 중복되지 않은 고유한 영화는 총 3628개
* 중복되지 않은 고유한 유저는 총 6039명

In [8]:
# 데이터 합치기
ratings_ = ratings.join(movies.set_index('movie_id'), on='movie_id')
del ratings_['genre']
del ratings_['timestamp']
ratings_.head()

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


In [9]:
best_mv_count = ratings_.groupby('title')['user_id'].count()
best_mv_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

# 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가

In [10]:
# 영화 찾기
movies[movies['title'].str.contains('Godfather', regex=False)]

,movie_id,title,genre
847,858,"Godfather, The (1972)",Action|Crime|Drama
1203,1221,"Godfather: Part II, The (1974)",Action|Crime|Drama
1954,2023,"Godfather: Part III, The (1990)",Action|Crime|Drama


In [11]:
# 내가 좋아하는 5개 영화 리스트 만들기
my_favorite_title = ['Godfather, The (1972)',
                     'Sixth Sense, The (1999)',
                     'Toy Story (1995)',
                     'Forrest Gump (1994)',
                     'Batman (1989)']
my_favorite_id = [858, 2762, 3114, 356, 592]

my_movielist = pd.DataFrame({'user_id': ['jinhwan']*5, 'movie_id': my_favorite_id, 'count': [5]*5, 'title': my_favorite_title})
my_movielist

,user_id,movie_id,count,title
0,jinhwan,858,5,"Godfather, The (1972)"
1,jinhwan,2762,5,"Sixth Sense, The (1999)"
2,jinhwan,3114,5,Toy Story (1995)
3,jinhwan,356,5,Forrest Gump (1994)
4,jinhwan,592,5,Batman (1989)


In [12]:
# ratings_에 추가
if not ratings_.isin({'user_id':['jinhwan']})['user_id'].any():
    ratings_ = ratings_.append(my_movielist)                     

ratings_.tail(10)

,user_id,movie_id,count,title
1000203,6040,1090,3,Platoon (1986)
1000205,6040,1094,5,"Crying Game, The (1992)"
1000206,6040,562,5,Welcome to the Dollhouse (1995)
1000207,6040,1096,4,Sophie's Choice (1982)
1000208,6040,1097,4,E.T. the Extra-Terrestrial (1982)
0,jinhwan,858,5,"Godfather, The (1972)"
1,jinhwan,2762,5,"Sixth Sense, The (1999)"
2,jinhwan,3114,5,Toy Story (1995)
3,jinhwan,356,5,Forrest Gump (1994)
4,jinhwan,592,5,Batman (1989)


# 4) CSR Matrix 만들기

In [13]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings_['user_id'].unique()
movie_unique = ratings_['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [14]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings_['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings_):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings_['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = ratings_['title'].map(movie_to_idx.get).dropna()
if len(temp_artist_data) == len(ratings_):
    print('title column indexing OK!!')
    ratings_['title'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

ratings_

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
0,6039,858,5,607
1,6039,2762,5,38
2,6039,3114,5,40
3,6039,356,5,160


In [15]:
# CSR Matrix 생성
num_user = ratings_['user_id'].nunique()
num_movie = ratings_['title'].nunique()

csr_data = csr_matrix((ratings_['count'], (ratings_.user_id, ratings_.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) AlternatingLeastSquares 모델 훈련

In [16]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악
* 훈련된 모델이 예측한 나의 선호도 파악
* 내가 선호하는 5가지 영화 중 하나
* 내가 선호하지 않는 영화

In [17]:
jinhwan, batman = user_to_idx['jinhwan'], movie_to_idx['Batman (1989)']
jinhwan_vector, batman_vector = als_model.user_factors[jinhwan], als_model.item_factors[batman]

In [18]:
jinhwan_vector

array([-0.08016422, -0.31433478, -0.28377458,  1.1748247 , -0.6443186 ,
       -0.36480337, -0.5733615 ,  0.21051183,  0.6563495 , -0.1609065 ,
       -0.4293284 , -0.89978045, -0.05263476,  0.03541632,  0.9668796 ,
        0.01757736,  0.04619829, -0.18629469,  0.30836284,  0.39242667,
        0.00473395, -0.21168894, -0.4225764 , -0.29097524, -0.18997972,
        1.0463707 ,  0.06904049, -0.53053176, -0.03785737,  0.38020784,
       -0.16323589, -0.12359643, -0.860347  , -0.14643912, -0.52740145,
        0.02077994,  0.8634127 ,  0.12857157, -0.47486162,  0.16660662,
       -0.20291786, -1.4977325 , -0.00959306, -0.29941723,  0.40344024,
        0.5300711 , -1.4593985 ,  0.03083185, -0.8493566 ,  0.4781036 ,
        0.04513239,  0.03856448, -0.2659251 ,  0.45631576,  0.3443353 ,
        0.11497069,  0.7913684 ,  0.3493981 ,  0.1641791 ,  0.51908785,
        1.1632252 , -0.43862438,  0.40247527,  0.5712524 ,  0.3968426 ,
       -0.47634   , -0.42398775, -0.41398972, -0.4415272 , -0.06

In [19]:
batman_vector

array([-0.01070347,  0.00643493,  0.03009223,  0.01365074, -0.02679911,
       -0.02078483, -0.01560608, -0.01151208,  0.02278595,  0.00701888,
       -0.00392668, -0.00904298,  0.01305003, -0.00280651,  0.04805545,
       -0.0105299 ,  0.00887822,  0.03638432,  0.00733519,  0.009588  ,
        0.01134573,  0.01508646,  0.01489545,  0.00636381,  0.01207307,
       -0.00450937,  0.01504163, -0.01712821,  0.00235907,  0.01962493,
        0.01489678,  0.0089905 , -0.01659449,  0.00648532, -0.00408041,
       -0.00666768,  0.01401787,  0.00576269,  0.00231534,  0.00152917,
        0.00564989,  0.0076234 , -0.01206359,  0.0153857 , -0.00281243,
       -0.00686285,  0.01354067,  0.01018972, -0.01988634,  0.00664165,
        0.00205289, -0.01205592,  0.02362691,  0.03647912,  0.00545329,
        0.00481645,  0.005079  , -0.00837463, -0.00311951,  0.00298135,
        0.01111122,  0.01281976, -0.0002852 ,  0.03527508, -0.00929858,
        0.00190806,  0.00621807,  0.00779285, -0.00169551,  0.00

In [20]:
# 선호하는 리스트에 있는 영화
np.dot(jinhwan_vector, batman_vector)

0.3336984

In [21]:
# 선호하는 리스트에 없는 영화
fargo = movie_to_idx['Fargo (1996)']
fargo_vector = als_model.item_factors[fargo]
np.dot(jinhwan_vector, fargo_vector)

0.13916315

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천

In [22]:
favorite_movie = 'Terminator, The (1984)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie   # (movie idx, 유사도) Tuple

[(200, 1.0000001),
 (651, 0.77666116),
 (194, 0.70882094),
 (865, 0.68229425),
 (193, 0.6747995),
 (92, 0.6040236),
 (124, 0.57535774),
 (928, 0.53792787),
 (680, 0.5224005),
 (117, 0.5094738),
 (882, 0.50056136),
 (62, 0.50027305),
 (172, 0.49189132),
 (44, 0.45817778),
 (120, 0.44599497)]

In [23]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Terminator, The (1984)',
 'Aliens (1986)',
 'Die Hard (1988)',
 'Predator (1987)',
 'Alien (1979)',
 'Terminator 2: Judgment Day (1991)',
 'Matrix, The (1999)',
 'Robocop (1987)',
 'Blade Runner (1982)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Mad Max 2 (a.k.a. The Road Warrior) (1981)',
 'Total Recall (1990)',
 'Indiana Jones and the Last Crusade (1989)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Raiders of the Lost Ark (1981)']

In [24]:
# 비슷한 영화를 알려주는 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [25]:
get_similar_movie('Ghostbusters (1984)')

['Ghostbusters (1984)',
 'Back to the Future (1985)',
 'Who Framed Roger Rabbit? (1988)',
 "Ferris Bueller's Day Off (1986)",
 'Big (1988)',
 'Fish Called Wanda, A (1988)',
 'Beetlejuice (1988)',
 'Young Frankenstein (1974)',
 'When Harry Met Sally... (1989)',
 'Out of Africa (1985)']

# 8) 내가 가장 좋아할 만한 영화를 추천

In [26]:
user = user_to_idx['jinhwan']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(380, 0.59691346),
 (50, 0.57397574),
 (64, 0.33454403),
 (110, 0.30128905),
 (4, 0.30065435),
 (435, 0.28612435),
 (154, 0.23205937),
 (172, 0.22650132),
 (126, 0.2222749),
 (170, 0.218175),
 (322, 0.21351202),
 (121, 0.19849297),
 (33, 0.19793594),
 (188, 0.1949763),
 (117, 0.19431195),
 (273, 0.19078754),
 (826, 0.19045243),
 (220, 0.18168643),
 (57, 0.18148226),
 (99, 0.18099923)]

In [27]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Godfather: Part II, The (1974)',
 'Toy Story 2 (1999)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Groundhog Day (1993)',
 "Bug's Life, A (1998)",
 'Godfather: Part III, The (1990)',
 'As Good As It Gets (1997)',
 'Indiana Jones and the Last Crusade (1989)',
 'Shakespeare in Love (1998)',
 'Being John Malkovich (1999)',
 'Babe (1995)',
 'Silence of the Lambs, The (1991)',
 'Aladdin (1992)',
 'Indiana Jones and the Temple of Doom (1984)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Fight Club (1999)',
 'Game, The (1997)',
 'Seven (Se7en) (1995)',
 'Untouchables, The (1987)',
 'American Beauty (1999)']

In [28]:
# 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인
recommended = movie_to_idx['Batman (1989)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Batman (1989)', 0.30873106803733147),
 ('Godfather, The (1972)', 0.022994271678067765),
 ('Forrest Gump (1994)', 0.0031796417643503447),
 ('Sixth Sense, The (1999)', 0.0005154186069720254),
 ('Toy Story (1995)', -0.006767164588709347)]